In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers, Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, Callback

# telegram
import telegram

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import os
import sys
import datetime
import time
import math

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


### Code for info dataframe reset 

In [2]:
cols = ['Model', "PId" ,'Class', "Phone" ,'rmse_mean', 'rmse_std', 'euc_mean', 
        'euc_std', 'XErr_mean', 'XErr_std', 'YErr_mean', 'YErr_std']
df_ = pd.DataFrame(columns=cols)
df_.to_pickle(str(Path.home())+"/data/model_info_cross.pkl")

In [3]:
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth=True

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.2

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [4]:
###############################################
#Returns screen width, height, pixel size (mm)#
###############################################
def get_screen_size(device):
    return {
        'S3Mini' : np.array([480, 800, 0.1089]),
        'S4' : np.array([1080, 1920, 0.0577]), 
        'N5X': np.array([1080, 1920, 0.06]),
        'N6' : np.array([1440, 2560, 0.05109])
    }.get(device, None)

def normalizeWidth(row):
    smartphone = row.Phone
    return row.XPress / get_screen_size(smartphone)[0]

def normalizeHeight(row):
    smartphone = row.Phone    
    return row.YPress / get_screen_size(smartphone)[1]

########################
#Metrics for evaluation#
########################
def eucInMM(y_true, y_pred):
            return (K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))) * constant_pixels
def euc(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))#
def eucStdMM(y_true, y_pred):
    return K.std(K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1))) * constant_pixels
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

####################################
#Metrics for dataframe calculations#
####################################
def calculate_eucMM(row):
    return math.sqrt((row['XPred'] - row['X'])**2 + (row['YPred'] - row['Y'])**2) * smartphone_values[2]

def calculate_rmseMM(row):
    return math.sqrt(mean_squared_error((row['X'],row['Y']), (row['XPred'],row['YPred']))) * smartphone_values[2]    

def calculate_x_error(row):
    return (row["XPred"] - row["X"]) * smartphone_values[2]

def calculate_y_error(row):
    return (row["YPred"] - row["Y"]) * smartphone_values[2]

In [5]:
smartphones = ["S3Mini", "S4", "N5X", "N6"]

In [6]:
%%time

batch_size = 400

df = pd.read_pickle("../../data/step03.pkl")
directory = str(Path.home())+"/data/cross/"

############################################################
#Generate list holding paths to all cross-validation models#
############################################################
modellist = []
for subdir, dirs, files in os.walk(directory):
    for file in files:
        modellist.append(os.path.join(subdir, file))

#####################################################
#Process every partcipant model for every smartphone#
#####################################################
for model_path in modellist:
    pid = int(model_path.split("_")[1].split("-")[2])
    model_info = model_path.split("/")[-1].split(".")[0]
    print("Start predicition for participant " + str(pid))
    for s, smartphone in enumerate(smartphones):
        smartphone_values = get_screen_size(smartphone)
        
        df_temp = df[(df.Phone == smartphone)].copy(deep = True)
        ###################################################
        #Normalize X and Y labels and shuffle participants#
        ###################################################
        df_temp.XPress = df_temp.apply(lambda x: normalizeWidth(x), axis=1)
        df_temp.YPress = df_temp.apply(lambda x: normalizeHeight(x), axis=1)
        df_temp.Sensor = df_temp.Sensor.apply(lambda x: x.reshape(-1, 6, 1))

        shape = df_temp.Sensor.iloc[0].shape

        #######################################################
        #Generate participant specific test set for validation#
        #######################################################
        
        dfTest = df_temp[df_temp.PId.isin([pid])]
        
        test_x = np.concatenate(dfTest.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
        test_y = dfTest[['XPress', 'YPress']].values

        #############################################################
        #Add zeros to dataset to make length dividable by batch size#
        #############################################################
        test_offset = batch_size - (len(test_x) % batch_size)
        test_x_zeros = np.zeros((test_offset, 100, 6, 1))
        test_y_zeros = np.zeros((test_offset, 2), dtype = int)
        test_y = np.concatenate([test_y, test_y_zeros])
        test_x = np.concatenate([test_x, test_x_zeros])
    
        ##################################################################################
        #Load the model, predict with it, generate both train and test prediction history#
        ##################################################################################
        print("Start "+smartphone+" predicition for participant " + str(pid))
        tf.get_default_graph()
        model = load_model(model_path, custom_objects={'euc':euc, 'rmse':rmse})
        optimizer = optimizers.Adam(lr = 0.001, decay = 1e-6)
        model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mae', euc, rmse])
        
        hist_test = model.predict(test_x, batch_size=batch_size, verbose=0, steps=None)
        ##################################################################
        #Generate DATAFRAME with all predicitions from test validation   #
        #These are used later for filling the model information dataframe# 
        ##################################################################
        df_test = pd.DataFrame(np.concatenate([hist_test[:-test_offset], test_y[:-test_offset]], axis = 1), 
                               columns = ["XPred","YPred", "X", "Y"])
        
        ####################
        #Undo normalization#
        ####################
        df_test.XPred = df_test.XPred * smartphone_values[0]
        df_test.YPred = df_test.YPred * smartphone_values[1]
        df_test.X = df_test.X * smartphone_values[0]
        df_test.Y = df_test.Y * smartphone_values[1]
        
        ###############################
        #Change column type to integer#
        ###############################
        df_test.XPred = df_test.XPred.astype(int)
        df_test.YPred = df_test.YPred.astype(int)
        df_test.X = df_test.X.astype(int)
        df_test.Y = df_test.Y.astype(int)
        
        ###################################################
        #Calculate RMSE, euclidean distance, x and y error#
        ###################################################
        df_test['rmse'] = df_test.apply(calculate_rmseMM, axis = 1) 
        df_test['euc'] = df_test.apply(calculate_eucMM, axis = 1)
        df_test['XErr'] = df_test.apply(calculate_x_error, axis = 1)
        df_test['YErr'] = df_test.apply(calculate_y_error, axis = 1)
        
        ##########################################################################################
        #Read cross validation pickle, create dict from columns, fill it and append it to info df#
        ##########################################################################################
        df_info = pd.read_pickle(str(Path.home())+"/data/model_info_cross.pkl")
        cols = df_info.columns.tolist()
        run_info = dict.fromkeys(cols)
        
        run_info['Model'] = model_info
        run_info['PId'] = pid
        run_info['Phone'] = smartphone
        run_info['Class'] = model_info.split("_")[1]
        run_info['rmse_mean'] = df_test.rmse.mean()
        run_info['rmse_std'] = df_test.rmse.std()
        run_info['euc_mean'] = df_test.euc.mean()
        run_info['euc_std'] = df_test.euc.std()
        run_info['XErr_mean'] = df_test.XErr.mean()
        run_info['XErr_std'] = df_test.XErr.std()
        run_info['YErr_mean'] = df_test.YErr.mean()
        run_info['YErr_std'] = df_test.YErr.std()
        
        df_info = df_info.append(run_info, ignore_index=True)
        
        df_info.to_pickle(str(Path.home())+"/data/model_info_cross.pkl")
    print("Participant "+ str(pid) +" done!")

Start predicition for participant 1
Start S3Mini predicition for participant 1
Start S4 predicition for participant 1
Start N5X predicition for participant 1
Start N6 predicition for participant 1
Participant 1 done!
Start predicition for participant 2
Start S3Mini predicition for participant 2
Start S4 predicition for participant 2
Start N5X predicition for participant 2
Start N6 predicition for participant 2
Participant 2 done!
Start predicition for participant 12
Start S3Mini predicition for participant 12
Start S4 predicition for participant 12
Start N5X predicition for participant 12
Start N6 predicition for participant 12
Participant 12 done!
Start predicition for participant 19
Start S3Mini predicition for participant 19
Start S4 predicition for participant 19
Start N5X predicition for participant 19
Start N6 predicition for participant 19
Participant 19 done!
Start predicition for participant 7
Start S3Mini predicition for participant 7
Start S4 predicition for participant 7
St

In [7]:
df_info

,Model,PId,Class,Phone,rmse_mean,rmse_std,euc_mean,euc_std,XErr_mean,XErr_std,YErr_mean,YErr_std
0,IMU_general-cross-1_20190201_214352,1,general-cross-1,S3Mini,8.152289,4.491626,11.529077,6.352118,-1.615350,9.443181,-2.994498,8.532322
1,IMU_general-cross-1_20190201_214352,1,general-cross-1,S4,11.711693,6.931287,16.562836,9.802320,-3.803392,12.160815,-1.169494,14.397703
2,IMU_general-cross-1_20190201_214352,1,general-cross-1,N5X,13.701891,8.021877,19.377400,11.344647,-3.621667,13.478967,-3.748889,17.208707
3,IMU_general-cross-1_20190201_214352,1,general-cross-1,N6,15.343104,9.990213,21.698426,14.128295,-4.892459,14.410883,-1.646588,20.907894
4,IMU_general-cross-2_20190201_214352,2,general-cross-2,S3Mini,7.992697,4.087730,11.303381,5.780923,-0.499629,7.950336,-0.270233,9.897004
5,IMU_general-cross-2_20190201_214352,2,general-cross-2,S4,10.614161,5.732846,15.010690,8.107468,-3.844930,10.292839,-5.623613,11.794742
6,IMU_general-cross-2_20190201_214352,2,general-cross-2,N5X,12.185576,6.125427,17.233007,8.662662,-5.483889,12.908357,-3.420000,12.814650
7,IMU_general-cross-2_20190201_214352,2,general-cross-2,N6,9.710526,5.237367,13.732758,7.406755,-4.394686,9.141735,-1.776205,11.738595
8,IMU_general-cross-12_20190201_214352,12,general-cross-12,S3Mini,8.624586,4.810000,12.197006,6.802367,-0.956404,8.187841,-2.254885,11.060310
9,IMU_general-cross-12_20190201_214352,12,general-cross-12,S4,10.170809,6.664978,14.383696,9.425702,-2.081341,8.778835,-3.110057,14.321652


In [10]:
for phone in smartphones:
    df_tmp = df_info[df_info.Phone == phone]
    cols = ['XErr_mean','XErr_std','YErr_mean','YErr_std']
    for col in cols:
        df_tmp[col] = df_tmp[col].abs()
        print(phone,col, df_tmp[col].mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


S3Mini XErr_mean 1.6361468750000001
S3Mini XErr_std 8.899383744669786
S3Mini YErr_mean 2.575699270833333
S3Mini YErr_std 10.527047842931735
S4 XErr_mean 1.4673630902777777
S4 XErr_std 10.571664964570562
S4 YErr_mean 3.358433842592592
S4 YErr_std 14.088398456045649
N5X XErr_mean 1.8144513888888891
N5X XErr_std 11.759302503383259
N5X YErr_mean 3.949881944444445
N5X YErr_std 14.542353803740085
N6 XErr_mean 2.2279438368055557
N6 XErr_std 10.720763440268614
N6 YErr_mean 3.7329937395833332
N6 YErr_std 16.10120240336039


In [22]:
print(df_info.groupby(["PId"])["euc_mean"].describe().sort_values("mean").head(15).index.unique())
df_info.groupby(["PId"])["euc_mean"].describe().sort_values("mean").head(15)

Int64Index([9, 15, 16, 19, 2, 13, 4, 14, 18, 12, 11, 7, 5, 17, 20], dtype='int64', name='PId')


,count,mean,std,min,25%,50%,75%,max
PId,,,,,,,,
9,4.0,12.999383,1.001792,11.725337,12.605027,13.060289,13.454645,14.151615
15,4.0,13.173712,1.372590,11.268835,12.839553,13.443277,13.777436,14.539462
16,4.0,13.207840,2.302137,10.724541,11.983994,12.949147,14.172992,16.208524
19,4.0,13.328701,2.205005,10.588546,12.631415,13.369509,14.066795,15.987241
2,4.0,14.319959,2.477062,11.303381,13.125414,14.371724,15.566270,17.233007
13,4.0,14.476140,1.739316,12.255610,13.515966,14.823084,15.783258,16.002782
4,4.0,14.692602,2.588424,11.503053,13.168274,15.041731,16.566059,17.183895
14,4.0,14.848993,1.396503,13.176406,13.957568,15.095399,15.986824,16.028768
18,4.0,14.948900,2.041620,12.227296,13.994302,15.380399,16.334997,16.807503


In [19]:
df_info.groupby(["PId"])["euc_mean"].describe().sort_values("mean").tail(5)

,count,mean,std,min,25%,50%,75%,max
PId,,,,,,,,
1,4.0,17.291935,4.378297,11.529077,15.304396,17.970118,19.957657,21.698426
6,4.0,17.895498,2.452439,14.841991,16.444922,18.422907,19.873483,19.894188
10,4.0,18.132698,2.145989,15.171930,17.244435,18.769251,19.657515,19.820360
3,4.0,18.145689,3.494456,13.048362,17.363398,19.372475,20.154766,20.789446
8,4.0,18.324364,2.523514,14.661056,17.985239,19.096158,19.435283,20.444082
